# Optimización del Cultivo en una Granja

## Contexto

En una granja de Segovia se cultivan diferentes tipos de vegetales durante la temporada de verano. Los cultivos deben planificarse teniendo en cuenta la disponibilidad de agua, fertilizantes y superficie cultivable, con el objetivo de maximizar el beneficio total.

Los cultivos disponibles son:

1. **Tomates**.
2. **Pimientos**.
3. **Lechugas**.
4. **Calabacines**.
5. **Berenjenas**.
6. **Zanahorias**.

---

## Datos del Problema

| Cultivo      | Beneficio por Hectárea (€) | Agua (m³/hectárea) | Fertilizantes (kg/hectárea) | Superficie mínima requerida (hectáreas) |
|--------------|----------------------------|---------------------|-----------------------------|------------------------------------------|
| Tomates      | 2000                       | 300                 | 50                          | 10                                       |
| Pimientos    | 2500                       | 400                 | 60                          | 15                                       |
| Lechugas     | 1500                       | 200                 | 40                          | 5                                        |
| Calabacines  | 1800                       | 350                 | 55                          | 8                                        |
| Berenjenas   | 2200                       | 450                 | 70                          | 10                                       |
| Zanahorias   | 1700                       | 300                 | 50                          | 7                                        |

### Recursos Disponibles

- **Superficie total disponible**: 100 hectáreas.
- **Agua disponible**: 30,000 m³.
- **Fertilizantes disponibles**: 5,000 kg.

---

## Formulación del Problema

Maximizar:  
$$ 
Z = 2000X_1 + 2500X_2 + 1500X_3 + 1800X_4 + 2200X_5 + 1700X_6 
$$

Sujeto a las siguientes restricciones:  
$$
\begin{aligned}
    X_1 + X_2 + X_3 + X_4 + X_5 + X_6 &\leq 100 \\
    300X_1 + 400X_2 + 200X_3 + 350X_4 + 450X_5 + 300X_6 &\leq 30,000 \\
    50X_1 + 60X_2 + 40X_3 + 55X_4 + 70X_5 + 50X_6 &\leq 5,000 \\
    X_1 &\geq 10 \\
    X_2 &\geq 15 \\
    X_3 &\geq 5 \\
    X_4 &\geq 8 \\
    X_5 &\geq 10 \\
    X_6 &\geq 7 \\
    X_1, X_2, X_3, X_4, X_5, X_6 &\geq 0
\end{aligned}
$$

---

## Interpretación

El objetivo del problema es determinar cuántas hectáreas se deben dedicar a cada tipo de cultivo para maximizar el beneficio total, respetando las limitaciones de recursos y las exigencias mínimas de superficie para cada tipo de vegetal.

In [54]:
from pyomo.environ import *

# Crear el modelo
model = ConcreteModel()

# Variables de decisión (hectáreas dedicadas a cada cultivo)
model.x1 = Var(within=NonNegativeReals)  # Tomates
model.x2 = Var(within=NonNegativeReals)  # Pimientos
model.x3 = Var(within=NonNegativeReals)  # Lechugas
model.x4 = Var(within=NonNegativeReals)  # Calabacines
model.x5 = Var(within=NonNegativeReals)  # Berenjenas
model.x6 = Var(within=NonNegativeReals)  # Zanahorias

# Función objetivo: Maximizar beneficios
model.obj = Objective(
    expr=2000*model.x1 + 2500*model.x2 + 1500*model.x3 +
         1800*model.x4 + 2200*model.x5 + 1700*model.x6,
    sense=maximize
)

# Restricciones
model.constr1 = Constraint(expr=model.x1 + model.x2 + model.x3 +
                           model.x4 + model.x5 + model.x6 <= 100)
model.constr2 = Constraint(expr=300*model.x1 + 400*model.x2 + 200*model.x3 +
                           350*model.x4 + 450*model.x5 + 300*model.x6 <= 30000)
model.constr3 = Constraint(expr=50*model.x1 + 60*model.x2 + 40*model.x3 +
                           55*model.x4 + 70*model.x5 + 50*model.x6 <= 5000)
model.constr4 = Constraint(expr=model.x1 >= 10)
model.constr5 = Constraint(expr=model.x2 >= 15)
model.constr6 = Constraint(expr=model.x3 >= 5)
model.constr7 = Constraint(expr=model.x4 >= 8)
model.constr8 = Constraint(expr=model.x5 >= 10)
model.constr9 = Constraint(expr=model.x6 >= 7)

# Resolver el modelo
solver = SolverFactory('glpk')
result = solver.solve(model)

# Mostrar resultados
print(f"Estado de la solución: {result.solver.status}")
print(f"Estado de la optimización: {result.solver.termination_condition}\n")

print(f"Ganancia máxima: {model.obj():.2f} €\n")
print(f"Producción óptima:")
print(f"Tomates: {model.x1():.2f} hectáreas")
print(f"Pimientos: {model.x2():.2f} hectáreas")
print(f"Lechugas: {model.x3():.2f} hectáreas")
print(f"Calabacines: {model.x4():.2f} hectáreas")
print(f"Berenjenas: {model.x5():.2f} hectáreas")
print(f"Zanahorias: {model.x6():.2f} hectáreas")

Estado de la solución: ok
Estado de la optimización: optimal

Ganancia máxima: 187550.00 €

Producción óptima:
Tomates: 31.00 hectáreas
Pimientos: 15.00 hectáreas
Lechugas: 26.50 hectáreas
Calabacines: 8.00 hectáreas
Berenjenas: 10.00 hectáreas
Zanahorias: 7.00 hectáreas


# Problema Dual: Optimización del Cultivo en una Granja

## Contexto del Problema Dual

En el problema dual, buscamos determinar el valor asociado a cada restricción del problema primal. Esto implica asignar un precio sombra a los recursos disponibles (agua, fertilizantes, superficie total) y al cumplimiento de las superficies mínimas requeridas por cada cultivo. El objetivo es minimizar el coste total de los recursos utilizados mientras se asegura que las condiciones de beneficio por hectárea se cumplen para cada cultivo.

---

## Formulación del Problema Dual

Minimizar:  
$$
W = 100Y_1 + 30,000Y_2 + 5,000Y_3 + 10Y_4 + 15Y_5 + 5Y_6 + 8Y_7 + 10Y_8 + 7Y_9
$$

Sujeto a las siguientes restricciones:  
$$
\begin{aligned}
    Y_1 + 300Y_2 + 50Y_3 + Y_4 &\geq 2000 & \text{(Tomates)} \\
    Y_1 + 400Y_2 + 60Y_3 + Y_5 &\geq 2500 & \text{(Pimientos)} \\
    Y_1 + 200Y_2 + 40Y_3 + Y_6 &\geq 1500 & \text{(Lechugas)} \\
    Y_1 + 350Y_2 + 55Y_3 + Y_7 &\geq 1800 & \text{(Calabacines)} \\
    Y_1 + 450Y_2 + 70Y_3 + Y_8 &\geq 2200 & \text{(Berenjenas)} \\
    Y_1 + 300Y_2 + 50Y_3 + Y_9 &\geq 1700 & \text{(Zanahorias)} \\
    Y_1, Y_2, Y_3, Y_4, Y_5, Y_6, Y_7, Y_8, Y_9 &\geq 0
\end{aligned}
$$

---

## Explicación de las Variables

- $Y_1$: Precio sombra por hectárea de superficie cultivable.
- $Y_2$: Precio sombra por m³ de agua disponible.
- $Y_3$: Precio sombra por kg de fertilizantes disponibles.
- $Y_4$: Valor asociado al cumplimiento de la superficie mínima para tomates.
- $Y_5$: Valor asociado al cumplimiento de la superficie mínima para pimientos.
- $Y_6$: Valor asociado al cumplimiento de la superficie mínima para lechugas.
- $Y_7$: Valor asociado al cumplimiento de la superficie mínima para calabacines.
- $Y_8$: Valor asociado al cumplimiento de la superficie mínima para berenjenas.
- $Y_9$: Valor asociado al cumplimiento de la superficie mínima para zanahorias.

---

## Interpretación

El problema dual nos ayuda a determinar el valor económico de los recursos y restricciones del problema primal. Cada variable $Y_i$ representa cuánto aumentaría el coste total si se incrementara la disponibilidad de recursos o se modificaran las restricciones mínimas en el primal. Esto permite evaluar la sensibilidad del sistema y priorizar inversiones en recursos o ajustes en los cultivos.

In [55]:
from pyomo.environ import *

# Crear el modelo
dual_model = ConcreteModel()

# Variables duales
dual_model.y1 = Var(within=NonNegativeReals)  # Precio sombra para superficie cultivable
dual_model.y2 = Var(within=NonNegativeReals)  # Precio sombra para agua
dual_model.y3 = Var(within=NonNegativeReals)  # Precio sombra para fertilizantes
dual_model.y4 = Var(within=NonNegativeReals)  # Precio sombra para superficie mínima de tomates
dual_model.y5 = Var(within=NonNegativeReals)  # Precio sombra para superficie mínima de pimientos
dual_model.y6 = Var(within=NonNegativeReals)  # Precio sombra para superficie mínima de lechugas
dual_model.y7 = Var(within=NonNegativeReals)  # Precio sombra para superficie mínima de calabacines
dual_model.y8 = Var(within=NonNegativeReals)  # Precio sombra para superficie mínima de berenjenas
dual_model.y9 = Var(within=NonNegativeReals)  # Precio sombra para superficie mínima de zanahorias

# Función objetivo: Minimizar el coste total de recursos y restricciones
dual_model.obj = Objective(
    expr=100*dual_model.y1 + 30000*dual_model.y2 + 5000*dual_model.y3 +
         10*dual_model.y4 + 15*dual_model.y5 + 5*dual_model.y6 +
         8*dual_model.y7 + 10*dual_model.y8 + 7*dual_model.y9,
    sense=minimize
)

# Restricciones duales: Asegurar que el coste asignado a cada cultivo no sea menor que su beneficio
dual_model.constr1 = Constraint(expr=dual_model.y1 + 300*dual_model.y2 + 50*dual_model.y3 + dual_model.y4 >= 2000)  # Tomates
dual_model.constr2 = Constraint(expr=dual_model.y1 + 400*dual_model.y2 + 60*dual_model.y3 + dual_model.y5 >= 2500)  # Pimientos
dual_model.constr3 = Constraint(expr=dual_model.y1 + 200*dual_model.y2 + 40*dual_model.y3 + dual_model.y6 >= 1500)  # Lechugas
dual_model.constr4 = Constraint(expr=dual_model.y1 + 350*dual_model.y2 + 55*dual_model.y3 + dual_model.y7 >= 1800)  # Calabacines
dual_model.constr5 = Constraint(expr=dual_model.y1 + 450*dual_model.y2 + 70*dual_model.y3 + dual_model.y8 >= 2200)  # Berenjenas
dual_model.constr6 = Constraint(expr=dual_model.y1 + 300*dual_model.y2 + 50*dual_model.y3 + dual_model.y9 >= 1700)  # Zanahorias

# Resolver el modelo
solver = SolverFactory('glpk')
result = solver.solve(dual_model)

# Mostrar resultados
print(f"Estado de la solución: {result.solver.status}")
print(f"Estado de la optimización: {result.solver.termination_condition}\n")

print(f"Coste total mínimo: {dual_model.obj():.2f} €\n")
print(f"Precios sombra (valores duales):")
print(f"Superficie cultivable (Y1): {dual_model.y1():.2f} €/hectárea")
print(f"Agua (Y2): {dual_model.y2():.2f} €/m³")
print(f"Fertilizantes (Y3): {dual_model.y3():.2f} €/kg")
print(f"Superficie mínima de tomates (Y4): {dual_model.y4():.2f} €")
print(f"Superficie mínima de pimientos (Y5): {dual_model.y5():.2f} €")
print(f"Superficie mínima de lechugas (Y6): {dual_model.y6():.2f} €")
print(f"Superficie mínima de calabacines (Y7): {dual_model.y7():.2f} €")
print(f"Superficie mínima de berenjenas (Y8): {dual_model.y8():.2f} €")
print(f"Superficie mínima de zanahorias (Y9): {dual_model.y9():.2f} €")

Estado de la solución: ok
Estado de la optimización: optimal

Coste total mínimo: 113300.00 €

Precios sombra (valores duales):
Superficie cultivable (Y1): 0.00 €/hectárea
Agua (Y2): 0.00 €/m³
Fertilizantes (Y3): 0.00 €/kg
Superficie mínima de tomates (Y4): 2000.00 €
Superficie mínima de pimientos (Y5): 2500.00 €
Superficie mínima de lechugas (Y6): 1500.00 €
Superficie mínima de calabacines (Y7): 1800.00 €
Superficie mínima de berenjenas (Y8): 2200.00 €
Superficie mínima de zanahorias (Y9): 1700.00 €


## Resultados del Problema Dual

### Coste Total Mínimo

El coste total mínimo obtenido es de **113,300 €**, que corresponde al valor óptimo del problema dual. Este valor indica el coste total asociado a satisfacer las demandas mínimas de superficie cultivable, agua, fertilizantes y las superficies mínimas requeridas para cada cultivo.

### Precios Sombra

Los precios sombra ($Y_i$) :



| Recurso / Restricción               | Precio Sombra (€) |
|-------------------------------------|-------------------|
| Superficie cultivable ($Y_1$)   | 0.00              |
| Agua ($Y_2$)                    | 0.00              |
| Fertilizantes ($Y_3$)           | 0.00              |
| Superficie mínima de tomates ($Y_4$) | 2,000.00        |
| Superficie mínima de pimientos ($Y_5$) | 2,500.00        |
| Superficie mínima de lechugas ($Y_6$)  | 1,500.00        |
| Superficie mínima de calabacines ($Y_7$) | 1,800.00        |
| Superficie mínima de berenjenas ($Y_8$) | 2,200.00        |
| Superficie mínima de zanahorias ($Y_9$) | 1,700.00        |

---

## Interpretación de los Resultados

### Recursos Sin Escasez

Los precios sombra asociados a los recursos básicos (superficie cultivable, agua y fertilizantes) son **0**. Esto significa que no se alcanzó el límite total de estos recursos en la solución óptima del primal. En otras palabras, aumentar la cantidad de estos recursos no tendría un impacto significativo en el beneficio total, ya que no son cuellos de botella.

### Restricciones Activas

Los precios sombra positivos ($Y_4$ a $Y_9$) están asociados a las superficies mínimas requeridas para cada cultivo. Esto indica que estas restricciones son activas en la solución óptima y, por lo tanto, están limitando el beneficio total. 

#### Interpretación de los Precios Sombra Positivos

- **Superficie mínima de tomates ($Y_4$):** Cada hectárea adicional para tomates contribuiría con 2,000 € al coste total del problema dual.
- **Superficie mínima de pimientos ($Y_5$):** Este cultivo tiene el mayor precio sombra, con 2,500 €. Incrementar la superficie dedicada a pimientos tendría un impacto considerable en la solución óptima.
- **Lechugas, calabacines, berenjenas y zanahorias ($Y_6$ a $Y_9$):** Cada una de estas restricciones tiene un precio sombra positivo, lo que implica que cumplir con sus superficies mínimas requeridas es esencial para mantener la solución óptima.

### Relación con el Primal

El valor óptimo del problema dual (113,300 €) es consistente con el beneficio total obtenido en el problema primal (187,550 €), según el **Teorema de la Dualidad Fuerte**, que garantiza que ambos valores deben coincidir si la solución es óptima.

---

## Conclusiones

1. **Recursos disponibles:** Los recursos totales (superficie cultivable, agua y fertilizantes) no son limitantes en la solución óptima, lo que sugiere que hay capacidad adicional para producir más cultivos si fuera necesario.

2. **Restricciones activas:** Las restricciones de superficies mínimas requeridas para cada cultivo son críticas en el problema primal. Esto se refleja en los precios sombra positivos, que representan el valor económico de cumplir con estas restricciones.

3. **Optimización del sistema:** Este análisis sugiere que, para incrementar el beneficio total, sería más valioso relajar las restricciones de superficie mínima en cultivos como pimientos y berenjenas, ya que tienen los precios sombra más altos.

El análisis del problema dual proporciona una visión complementaria del problema primal, permitiendo identificar oportunidades para mejorar la planificación de recursos y cultivos en temporadas futuras.

# Grafo Primal

---

In [56]:
#ejecutar 
import plotly.graph_objects as go
import networkx as nx

# Crear el grafo para el problema primal
G_primal = nx.DiGraph()

# Agregar nodos (recursos y cultivos)
recursos = ["Superficie Total", "Agua", "Fertilizantes"]
cultivos = ["Tomates", "Pimientos", "Lechugas", "Calabacines", "Berenjenas", "Zanahorias"]

nodos_primal = recursos + cultivos
G_primal.add_nodes_from(nodos_primal)

# Agregar aristas (relaciones entre recursos y cultivos)
aristas_primal = [
    (r, c) for r in recursos for c in cultivos
]
G_primal.add_edges_from(aristas_primal)

# Resultados del problema primal
resultados_primal = {
    "Tomates": 31.00,
    "Pimientos": 15.00,
    "Lechugas": 26.50,
    "Calabacines": 8.00,
    "Berenjenas": 10.00,
    "Zanahorias": 7.00
}

# Posiciones manuales para nodos en capas
pos = {}
layer_distance = 1.5
for i, node in enumerate(recursos):
    pos[node] = (i * 2, 0, 0)  # Capa superior (recursos)
for i, node in enumerate(cultivos):
    pos[node] = (i * 2, -layer_distance, 0)  # Capa inferior (cultivos)

# Coordenadas para nodos y aristas
x_nodes = [pos[node][0] for node in nodos_primal]
y_nodes = [pos[node][1] for node in nodos_primal]
z_nodes = [pos[node][2] for node in nodos_primal]

x_edges = []
y_edges = []
z_edges = []

for edge in G_primal.edges():
    x_edges += [pos[edge[0]][0], pos[edge[1]][0], None]
    y_edges += [pos[edge[0]][1], pos[edge[1]][1], None]
    z_edges += [pos[edge[0]][2], pos[edge[1]][2], None]

# Crear etiquetas con los resultados del primal
labels = []
for node in G_primal.nodes():
    if node in resultados_primal:
        labels.append(f"{node}<br>{resultados_primal[node]} hectáreas")
    else:
        labels.append(node)

# Crear el grafo interactivo en 3D
fig = go.Figure()

# Agregar aristas
fig.add_trace(go.Scatter3d(
    x=x_edges, y=y_edges, z=z_edges,
    mode='lines',
    line=dict(color='gray', width=1),
    hoverinfo='none'
))

# Agregar nodos
fig.add_trace(go.Scatter3d(
    x=x_nodes, y=y_nodes, z=z_nodes,
    mode='markers+text',
    marker=dict(size=10, color='blue'),
    text=labels,  # Etiquetas con los resultados del primal
    textposition="top center",
    hovertemplate='%{text}<extra></extra>'
))

# Configurar el diseño
fig.update_layout(
    title="Grafo del Problema Primal en 3D con Distribución Lógica",
    scene=dict(
        xaxis=dict(title="Eje X", showgrid=False, zeroline=False),
        yaxis=dict(title="Eje Y", showgrid=False, zeroline=False),
        zaxis=dict(title="Eje Z", showgrid=False, zeroline=False)
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

fig.show(renderer='browser')    

# Grafo dual 
---

In [44]:
#ejecutar  
import plotly.graph_objects as go
import networkx as nx

# Crear el grafo para el problema dual
G_dual = nx.DiGraph()

# Nodos del dual
restricciones_dual = [
    "Tomates (2000)", "Pimientos (2500)", "Lechugas (1500)",
    "Calabacines (1800)", "Berenjenas (2200)", "Zanahorias (1700)"
]
variables_dual = [
    "Y1 (Superficie: 0 €/ha)", "Y2 (Agua: 0 €/m³)", "Y3 (Fertilizantes: 0 €/kg)",
    "Y4 (Tomates: 2000 €)", "Y5 (Pimientos: 2500 €)", "Y6 (Lechugas: 1500 €)",
    "Y7 (Calabacines: 1800 €)", "Y8 (Berenjenas: 2200 €)", "Y9 (Zanahorias: 1700 €)"
]

nodos_dual = restricciones_dual + variables_dual
G_dual.add_nodes_from(nodos_dual)

# Aristas del dual (relaciones entre restricciones y variables)
aristas_dual = [
    ("Tomates (2000)", "Y1 (Superficie: 0 €/ha)"),
    ("Tomates (2000)", "Y2 (Agua: 0 €/m³)"),
    ("Tomates (2000)", "Y3 (Fertilizantes: 0 €/kg)"),
    ("Tomates (2000)", "Y4 (Tomates: 2000 €)"),
    ("Pimientos (2500)", "Y1 (Superficie: 0 €/ha)"),
    ("Pimientos (2500)", "Y2 (Agua: 0 €/m³)"),
    ("Pimientos (2500)", "Y3 (Fertilizantes: 0 €/kg)"),
    ("Pimientos (2500)", "Y5 (Pimientos: 2500 €)"),
    ("Lechugas (1500)", "Y1 (Superficie: 0 €/ha)"),
    ("Lechugas (1500)", "Y2 (Agua: 0 €/m³)"),
    ("Lechugas (1500)", "Y3 (Fertilizantes: 0 €/kg)"),
    ("Lechugas (1500)", "Y6 (Lechugas: 1500 €)"),
    ("Calabacines (1800)", "Y1 (Superficie: 0 €/ha)"),
    ("Calabacines (1800)", "Y2 (Agua: 0 €/m³)"),
    ("Calabacines (1800)", "Y3 (Fertilizantes: 0 €/kg)"),
    ("Calabacines (1800)", "Y7 (Calabacines: 1800 €)"),
    ("Berenjenas (2200)", "Y1 (Superficie: 0 €/ha)"),
    ("Berenjenas (2200)", "Y2 (Agua: 0 €/m³)"),
    ("Berenjenas (2200)", "Y3 (Fertilizantes: 0 €/kg)"),
    ("Berenjenas (2200)", "Y8 (Berenjenas: 2200 €)"),
    ("Zanahorias (1700)", "Y1 (Superficie: 0 €/ha)"),
    ("Zanahorias (1700)", "Y2 (Agua: 0 €/m³)"),
    ("Zanahorias (1700)", "Y3 (Fertilizantes: 0 €/kg)"),
    ("Zanahorias (1700)", "Y9 (Zanahorias: 1700 €)")
]
G_dual.add_edges_from(aristas_dual)

# Posiciones manuales para nodos en capas
pos_dual = {}
layer_distance = 1.5
for i, node in enumerate(restricciones_dual):
    pos_dual[node] = (i * 2, 0, 0)  # Capa superior (restricciones)
for i, node in enumerate(variables_dual):
    pos_dual[node] = (i * 2, -layer_distance, 0)  # Capa inferior (variables)

# Coordenadas para nodos y aristas
x_nodes_dual = [pos_dual[node][0] for node in nodos_dual]
y_nodes_dual = [pos_dual[node][1] for node in nodos_dual]
z_nodes_dual = [pos_dual[node][2] for node in nodos_dual]

x_edges_dual = []
y_edges_dual = []
z_edges_dual = []

for edge in G_dual.edges():
    x_edges_dual += [pos_dual[edge[0]][0], pos_dual[edge[1]][0], None]
    y_edges_dual += [pos_dual[edge[0]][1], pos_dual[edge[1]][1], None]
    z_edges_dual += [pos_dual[edge[0]][2], pos_dual[edge[1]][2], None]

# Crear etiquetas para los nodos del dual
labels_dual = nodos_dual  # Los nombres de los nodos ya contienen la información

# Crear el grafo interactivo en 3D
fig_dual = go.Figure()

# Agregar aristas
fig_dual.add_trace(go.Scatter3d(
    x=x_edges_dual, y=y_edges_dual, z=z_edges_dual,
    mode='lines',
    line=dict(color='gray', width=1),
    hoverinfo='none'
))

# Agregar nodos
fig_dual.add_trace(go.Scatter3d(
    x=x_nodes_dual, y=y_nodes_dual, z=z_nodes_dual,
    mode='markers+text',
    marker=dict(size=10, color='orange'),
    text=labels_dual,  # Etiquetas con los precios sombra y restricciones
    textposition="top center",
    hovertemplate='%{text}<extra></extra>'
))

# Configurar el diseño
fig_dual.update_layout(
    title="Grafo del Problema Dual en 3D con Precios Sombra",
    scene=dict(
        xaxis=dict(title="Eje X", showgrid=False, zeroline=False),
        yaxis=dict(title="Eje Y", showgrid=False, zeroline=False),
        zaxis=dict(title="Eje Z", showgrid=False, zeroline=False)
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

fig_dual.show(renderer='browser')

# Conclusión Final y Análisis Matemático

A través del estudio de los problemas primal y dual, hemos optimizado el uso de recursos agrícolas en una granja para maximizar los beneficios bajo restricciones económicas y físicas. Basándonos en los fundamentos de la programación lineal descritos en [Linear Programming](https://webdiis.unizar.es/~jcampos/ab/material/7-LinearProgramming.pdf), presentamos las siguientes conclusiones.



## Resultados del Problema Dual

### Coste Total Mínimo

El coste total mínimo obtenido es de **113,300 €**, que corresponde al valor óptimo del problema dual. Este valor indica el coste total asociado a satisfacer las demandas mínimas de superficie cultivable, agua, fertilizantes y las superficies mínimas requeridas para cada cultivo.

### Precios Sombra

Los precios sombra ($Y_i$):

| Recurso / Restricción               | Precio Sombra (€) |
|-------------------------------------|-------------------|
| Superficie cultivable ($Y_1$)       | 0.00              |
| Agua ($Y_2$)                        | 0.00              |
| Fertilizantes ($Y_3$)               | 0.00              |
| Superficie mínima de tomates ($Y_4$)| 2,000.00          |
| Superficie mínima de pimientos ($Y_5$)| 2,500.00        |
| Superficie mínima de lechugas ($Y_6$)| 1,500.00        |
| Superficie mínima de calabacines ($Y_7$)| 1,800.00     |
| Superficie mínima de berenjenas ($Y_8$)| 2,200.00      |
| Superficie mínima de zanahorias ($Y_9$)| 1,700.00      |

---

## Interpretación de los Resultados

### Recursos Sin Escasez

Los precios sombra asociados a los recursos básicos (superficie cultivable, agua y fertilizantes) son **0**. Esto significa que no se alcanzó el límite total de estos recursos en la solución óptima del primal. En otras palabras, aumentar la cantidad de estos recursos no tendría un impacto significativo en el beneficio total, ya que no son cuellos de botella.

### Restricciones Activas

Los precios sombra positivos ($Y_4$ a $Y_9$) están asociados a las superficies mínimas requeridas para cada cultivo. Esto indica que estas restricciones son activas en la solución óptima y, por lo tanto, están limitando el beneficio total. 

#### Interpretación de los Precios Sombra Positivos

- **Superficie mínima de tomates ($Y_4$):** Cada hectárea adicional para tomates contribuiría con **2,000 €** al coste total del problema dual.
- **Superficie mínima de pimientos ($Y_5$):** Este cultivo tiene el mayor precio sombra, con **2,500 €**. Incrementar la superficie dedicada a pimientos tendría un impacto considerable en la solución óptima.
- **Lechugas, calabacines, berenjenas y zanahorias ($Y_6$ a $Y_9$):** Cada una de estas restricciones tiene un precio sombra positivo, lo que implica que cumplir con sus superficies mínimas requeridas es esencial para mantener la solución óptima.

### Relación con el Primal

El valor óptimo del problema dual (**113,300 €**) es consistente con el beneficio total obtenido en el problema primal (**187,550 €**), según el **Teorema de la Dualidad Fuerte**, que garantiza que ambos valores deben coincidir si la solución es óptima.

---

## Conclusiones

1. **Recursos disponibles:** Los recursos totales (superficie cultivable, agua y fertilizantes) no son limitantes en la solución óptima, lo que sugiere que hay capacidad adicional para producir más cultivos si fuera necesario.

2. **Restricciones activas:** Las restricciones de superficies mínimas requeridas para cada cultivo son críticas en el problema primal. Esto se refleja en los precios sombra positivos, que representan el valor económico de cumplir con estas restricciones.

3. **Optimización del sistema:** Este análisis sugiere que, para incrementar el beneficio total, sería más valioso relajar las restricciones de superficie mínima en cultivos como pimientos y berenjenas, ya que tienen los precios sombra más altos.

